#### First Let's understand the basic Catalog + Volume Feature of Databricks 
Filesystem Hierarchy of Volume in Databricks (DBFS)?<br>
Catalog -> /OurWorkspace/catalog/schema(database)/volume/folder/data files<br>
Tables Hierarchy of Databricks?<br>
Catalog -> /OurWorkspace/catalog/schema(database)/tables/data(dbfs filesystem/some other filesystems)<br>

####If we need to create schema/volume/folder programatically, follow the below steps

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS Catalog1;
USE CATALOG Catalog1;
CREATE SCHEMA IF NOT EXISTS Schema1;
USE SCHEMA Schema1;
CREATE VOLUME IF NOT EXISTS Volume1;

DROP CATALOG → Works only if catalog is fully empty<br>
DROP CATALOG CASCADE → Works even if catalog contains objects, because it force-deletes them

In [0]:
%sql
DROP CATALOG catalog1 CASCADE;

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS Catalog2;
CREATE DATABASE IF NOT EXISTS Catalog2.DataBase2;
CREATE VOLUME IF NOT EXISTS Catalog2.DataBase2.Volume2;

In [0]:
dbutils.fs.mkdirs("dbfs:///Volumes/workspace/default/healthcare/created_folder")

#Spark SQL<br>
###1.E(Extract) 
L(Load)<br>
Inbuilt libraries sources/targets & Inbuilt data Formats<br>
2. Bread & Butter (T(Transformation) A(Analytical))<br>

#Learn How to Create Dataframes from filesystem using different options
Download the data from the below drive url <br>
https://drive.google.com/drive/folders/1Tw7V9eBtUxy0xQMW38z3-bzWI_ewzLm6?usp=drive_link

###How Do We Write a Typical Spark Application (Core(Obsolute),SQL(Important),Streaming(Mid level important))
####Before we Create Dataframe/RDD, what is the prerequisite? We need to create spark session object by instantiating sparksession class (by default databricks did that if you create a notebook)

In [0]:
from pyspark.sql.session import SparkSession
print(spark)#already instantiated by databricks
spark1=SparkSession.builder.appName("Spark").getOrCreate()
print(spark1)#we instantiated

- Create a DBFS volume namely commondata and upload the above data in that volume.<br>
- What are other FS uri's available? file:///, hdfs:///, dbfs:///, gs:///, s3:///, adls:///, blob:///

###How to Read/Extract the data from the filesytem and load it into the distributed memory for further processing/load - using diffent methodologies/options from different sources(fs & db) and different builtin formats (csv/json/orc/parquet/delta/tables)

![Screenshot 2025-12-11 193250.png](./Screenshot 2025-12-11 193250.png "Screenshot 2025-12-11 193250.png")

In [0]:
%py
#If I do not specify any options in the CSV function, what defaults does Spark use?
#1. Delimiter->By default, Spark uses a comma , as the field separator.
#Example default: sep=","

#2. Column Names (Header)->If the CSV file does not have a header, Spark automatically assigns column names as:_c0, _c1, _c2, ..., _cn
#This is because the default option is:
#eader = false

#3. Data Types->By default, Spark reads all columns as string type because:
#inferSchema = false
#(Spark does not try to detect column types unless you enable it.

#we have more options to see further for above all

df1=spark.read.csv("dbfs:///Volumes/catalog2/database2/volume2/created_folder/custs")
df1.show(3)#display with produce output in a dataframe format
print(df1.printSchema())
display(df1)#display with produce output in a beautified table format, specific to databricks

%md
csv_df1=spark.read.csv
(path: PathOrPaths, schema: Optional[Union[StructType, str]]=None, sep: Optional[str]=None, encoding: Optional[str]=None, quote: Optional[str]=None, escape: Optional[str]=None, comment: Optional[str]=None, header: Optional[Union[bool, str]]=None, inferSchema: Optional[Union[bool, str]]=None, ignoreLeadingWhiteSpace: Optional[Union[bool, str]]=None, ignoreTrailingWhiteSpace: Optional[Union[bool, str]]=None, nullValue: Optional[str]=None, nanValue: Optional[str]=None, positiveInf: Optional[str]=None, negativeInf: Optional[str]=None, dateFormat: Optional[str]=None, timestampFormat: Optional[str]=None, maxColumns: Optional[Union[int, str]]=None, maxCharsPerColumn: Optional[Union[int, str]]=None, maxMalformedLogPerPartition: Optional[Union[int, str]]=None, mode: Optional[str]=None, columnNameOfCorruptRecord: Optional[str]=None, multiLine: Optional[Union[bool, str]]=None, charToEscapeQuoteEscaping: Optional[str]=None, samplingRatio: Optional[Union[float, str]]=None, enforceSchema: Optional[Union[bool, str]]=None, emptyValue: Optional[str]=None, locale: Optional[str]=None, lineSep: Optional[str]=None, pathGlobFilter: Optional[Union[bool, str]]=None, recursiveFileLookup: Optional[Union[bool, str]]=None, modifiedBefore: Optional[Union[bool, str]]=None, modifiedAfter: Optional[Union[bool, str]]=None, unescapedQuoteHandling: Optional[str]=None) -> "DataFrame"

Sample data of custs_1
4000001,Kristina,Chung,55,Pilot<br>
4000002,Paige,Chen,77,Teacher

Sample data of custs_header<br>
custid,fname,lname,age,profession<br>
4000001,Kristina,Chung,55,Pilot<br>
4000002,Paige,Chen,77,Teacher

In [0]:
%py
#1. Header Concepts (Either we have define the column names or we have to use the column names from the data)
#Important option is...
#By default, Spark assigns column names as _c0, _c1, _c2, ..., _cn because it does not treat the first row as a header (header = false).
#header=True means we are asking spark to take the first row as header and not as a data
df1=spark.read.csv("dbfs:///Volumes/catalog2/database2/volume2/created_folder/custs_header",header=True)
df1.show(3)
#df1.write.csv("/Volumes/workspace/default/healthcare/created_folder/copy.csv",header=True,mode="overwrite")
#By default it will use _c0,_c1..._cn it will apply as column headers, if we use toDF(colnames) we can define our own headers.
df2=spark.read.csv("dbfs:///Volumes/catalog2/database2/volume2/created_folder/cust_1.txt").toDF("id","fname","lname","age","prof")
df2.show(2)#display with produce output in a dataframe format
print(df2.printSchema())

In [0]:
#2. Printing Schema (equivalent to describe table)
df1.printSchema()
df2.printSchema()

In [0]:
#3. Inferring Schema 
# (Performance Consideration: Use this function causiously because it scans the entire data by immediately evaluating and executing
# hence, not good for large data or not good to use on the predefined schema dataset)
#sample data
#4004979,Tara,Drake,32,
#4004980,Earl,Hahn,34,Human resources assistant
#4004981,Don,Jones,THIRTY SIX,Lawyer
df2=spark.read.csv("dbfs:///Volumes/catalog2/database2/volume2/created_folder/cust_1.txt",inferSchema=True).toDF("id","fname","lname","age","prof")
df2.where("id in (4000009,4000010)").show(2)
print(df2.printSchema())

In [0]:
df2=spark.read.csv("dbfs:///Volumes/catalog2/database2/volume2/created_folder/cust_1.txt",inferSchema=True).toDF("id","fname","lname","age","prof")
print(df2.printSchema())

In [0]:
#4. Using delimiter or seperator option
csv_df1=spark.read.csv("dbfs:///Volumes/catalog2/database2/volume2/created_folder/delimiter_sample.txt",header=True,sep="~")
csv_df1.show(2)
csv_df1.printSchema()


In [0]:
#5. Using different options to create dataframe with csv and other module... (2 methodologies with 3 ways of creating dataframes)
csv_df1=spark.read.csv("dbfs:///Volumes/catalog2/database2/volume2/created_folder/delimiter_sample.txt",inferSchema=True, header=True,sep="~")
csv_df1.show(2)
#or another way of creating dataframe (from any sources whether builtin or external)...
#option can be used for 1 or 2 option...
spark.read.option("header","True").option("sep","~").option("inferSchema","True").format("csv").csv("dbfs:///Volumes/catalog2/database2/volume2/created_folder/delimiter_sample.txt").show(2)
#options can be used for multiple options in one function as a parameter...
spark.read.options(header="True",sep="~",inferSchema="True").format("csv").csv("dbfs:///Volumes/catalog2/database2/volume2/created_folder/delimiter_sample.txt").show(2)

### Generic way of read and load data into dataframe using fundamental options from built in sources (csv/orc/parquet/xml/json/table) (inferschema, header, sep)

In [0]:
csv_df1=spark.read.csv("dbfs:///Volumes/catalog2/database2/volume2/created_folder/delimiter_sample.txt",inferSchema=True,header=True,sep='~')
csv_df1.show(2)

### Generic way of read and load data into dataframe using extended options from external sources (bigquery/redshift/athena/synapse) (tmpfolder, access controls)
ex, I can use format("bigquery") but in using fundamental option it is not possible

In [0]:
#options can be used for multiple options in one function as a parameter...
csv_df3=spark.read.options(header="True",inferSchema="true",sep="~").format("csv").load("dbfs:///Volumes/catalog2/database2/volume2/created_folder/delimiter_sample.txt")
csv_df3.show(2)

### Reading data from multiple files

In [0]:
csv_df1=spark.read.csv("dbfs:///Volumes/catalog2/database2/volume2/created_folder/cust*",inferSchema=True,header=True,sep='~')
print(csv_df1.count())

In [0]:
csv_df1=spark.read.csv(path=["dbfs:///Volumes/catalog2/database2/volume2/created_folder/delimiter_sample.txt","dbfs:///Volumes/catalog2/database2/volume2/created_folder/cust_1.txt"],inferSchema=True,header=True,sep=',')
print(csv_df1.count())